#  Data collection & Munging

In dit notebook zal the ruwe datasets omgezet worden naar makkelijk bewerkbare data in een Pandas dataframe.

#### De XML dataset
In de XML dataset staat wel ontwikkeling zich afspelen op het veld. Je kunt denken aan bal winst of een goal. Ook wordt er bijghouden waar de ontwikkeling plaats vind. Bijvoorbeeld L2 of R1 (Zie afbeelding).
<img src="./voetbalveld-indeling.png" width=300>

##### XML structuur
De XML is als volgt op gebouwd:
```XML
<ALL_INSTANCES>
    <instance>
        <ID>1</ID>
        <start>4.3110428970</start>
        <end>40.9711876731</end>
        <code>TEG</code>
        <label>
            <group>Naar zone</group>
            <text>M2 TEG</text>
        </label>
    </instance>
</ALL_INSTANCES>
```
Er komen in ```<ALL_INSTANCES>``` meerdere instances voor.

Om een xml intelezen hebben van de XML library ElementTree nodig. Hiermee kunnen we makkelijk alle data uit de XML halen. We hebben ook pandas nodig omdat we pandas willen gebruiken voor de analyse.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

De eerste stap zal zijn om de XML dataset om te zetten naar een pandas dataframe. Hiervoor maken we een functie waarin we de root van de dataset kunnen meegeven. We maken er een functie van omdat we dit voor meerdere datasets willen doen. 
<br />
<br />
In de functie zullen door alle childeren van de root gaan (we zitten dan in ```<instance>```) en daarvan de subchilderen. voor elke subchild checken we of het een label tag is. Dit doen we omdat de label tag meer subchilderen heeft. Als dat niet zo is dan voegen we de tekst van de tag toe aan de record en als de tag naam niet in de header list zit voegen we die ook toe. Als de tag wel een label loopen we door de childeren van die label tag. Een instance tag kan een verschillende hoeveelheid label tags bevatten. Daarom moeten we meerdere headers toevoegen voor de label tags. 
<br />
<br />
Zoals eerder gezegd heeft niet elke instance een gelijke hoeveelheid labels. Bijvoorbeeld de instance met id twee heeft maar 1 label terwijl instance met id 1 een meer dan 10 labels heeft. We zullen dus null waarde krijgen. Dit is niet erg we zullen gewoon de null waarde invullen.  
<br />
<br />
Als dat allemaal gedaan is krijgen we een mooie pandas dataset waar we mee kunnen werken.

In [30]:
def xmlRootToDataframe(root):
    all_instances = root[1]
    if(root[1].tag == 'ALL_INSTANCES'):      
        all_instances = root[1]
    else:
        all_instances = root[0]
    all_records = []
    headers = []
    for i, child in enumerate(all_instances):
        current_label = 0
        record = []
        for subchild in child:
            label_name = ''
            if subchild.tag == 'label':
                subrecord = []
                label_name = subchild.tag+'.'+str(current_label)
                if label_name not in headers:
                        headers.append(label_name)
                for subsubchild in subchild:
                    subrecord.append(subsubchild.text)
                current_label += 1
                record.append(subrecord)
            else:
                record.append(subchild.text)
            if subchild.tag not in headers and subchild.tag not in 'label':
                headers.append(subchild.tag)
        all_records.append(record)
    return pd.DataFrame(all_records, columns=headers)


tree = ET.parse('datasets/new-xml/20160410 FC Utrecht - NEC-new.xml')
root = tree.getroot()
utrechtNec = xmlRootToDataframe(root)

tree= ET.parse('datasets/new-xml/20160417 NEC - Cambuur-new.xml')
root = tree.getroot()
necCambuur = xmlRootToDataframe(root)

tree= ET.parse('datasets/new-xml/20160420 PEC - NEC-new.xml')
root = tree.getroot()
pecNec = xmlRootToDataframe(root)

tree= ET.parse('datasets/new-xml/20160501 NEC - Roda JC-new.xml')
root = tree.getroot()
NecRoda = xmlRootToDataframe(root)

necCambuur.head()

,ID,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,...,label.46,label.47,label.48,label.49,label.50,label.51,label.52,label.53,label.54,label.55
0,1,18.6400000000,24.4700000000,AFTRAP WB,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,19.5113937674,37.5765464838,TEG,"[Naar zone, M2 TEG]","[Storen NEC, Druk]","[Storen NEC, Compact]","[Naar zone, M1 TEG]","[Aanvallen TEG, Lange pass]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
2,3,19.9194032119,27.9194032119,Start Jaimy,"[Naar zone, M2 TEG]","[Storen NEC, Druk]","[Storen NEC, Compact]",None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,4,21.5113937674,37.5765464838,Verdedigen,"[Naar zone, M2 TEG]","[Storen NEC, Druk]","[Storen NEC, Compact]","[Naar zone, M1 TEG]","[Aanvallen TEG, Lange pass]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,5,21.5432956698,28.2807977534,M2 TEG,"[Storen NEC, Druk]","[Storen NEC, Compact]","[Naar zone, M1 TEG]",None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [3]:
utrechtNec[['ID', 'start', 'end', 'code', 'label.0']].dtypes

ID         object
start      object
end        object
code       object
label.0    object
dtype: object

De datatypes van alle kolommen zijn nu objecten. Dit is niet idiaal en zal later beperkingen opleveren. laten we start en end numeric maken. Ook willen we categorical data maken van de code kolom. We maken de id kolom ook gelijk de index.
<br />
<br />
We beginnen de index veranderen daarna zetten we de datatypes om.

In [31]:
def changeDataTypes(df):

    df = df.set_index('ID')
    df.index.name = None

    df[['start','end']] = df[['start','end']].apply(pd.to_numeric)
    #df.code = df.code.astype('category')
    return df

utrechtNec = changeDataTypes(utrechtNec)
necCambuur = changeDataTypes(necCambuur)
pecNec = changeDataTypes(pecNec)
NecRoda = changeDataTypes(NecRoda)
 
print(utrechtNec[['start','end', 'code', 'label.0']].dtypes)

start      float64
end        float64
code        object
label.0     object
dtype: object


In [32]:
utrechtNec.head()

,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,label.6,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
1,4.311043,40.971188,TEG,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
2,6.215018,15.134795,M2 TEG,"[Naar zone, L2 TEG]",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,6.311043,40.971188,Start Jaimy,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,6.311043,40.971188,Verdedigen,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
5,8.220000,14.040000,AFTRAP WB,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Hebben nu een nog best wel ruwe dataset waar we niet veel mee kunnen. Laten we een dataframe maken die de positie van de bal en wie de bal bezit maken. In de code kolom staat welke 

In [27]:
def balBezitDF(df):
    balbezit = df.code.replace({'^L': '1', '^M':'2', '^R':'3'}, regex=True).str.extract('(?P<x>^[0-3])(?P<y>[0-4])(?P<bezit>.TEG)?', expand=True).dropna(how='all')
    tijd = df[['start', 'end']]
    
    balbezit = pd.concat([balbezit, df[df['code'].str.contains("Goal")]['code']], axis=1)
    balbezit[['x', 'y']] = balbezit[['x', 'y']].fillna(0)
    balbezit = pd.merge(tijd,balbezit, left_index = True, right_index=True)
    balbezit.bezit = balbezit.bezit.fillna('NEC')
    balbezit.code = balbezit.code.fillna('Nothing')
    
    balbezit[['x', 'y']] = balbezit[['x', 'y']].astype('int')
    balbezit.index = balbezit.index.astype('int')
    balbezit.sort_index(inplace=True)
    balbezit = balbezit.reset_index(drop=True)
    
    return balbezit

utrechtNecBB = balBezitDF(utrechtNec)
necCambuurBB = balBezitDF(necCambuur)
pecNecBB = balBezitDF(pecNec)
NecRodaBB = balBezitDF(NecRoda)

Nu we de data hebben omgezet naar een dataset waar we mee kunnen werken is het tijd om de data te exporteren naar een csv om het later te kunnen gebruiken in een ander notebook. We maken net zoals de vorige keren een functie aan die dit voor ons doet.

In [26]:
def exportDataframe(df, naam): 
    path = 'datasets/bewerkte-datasets/'+naam+'.csv'
    df.to_csv(path_or_buf=path, header=list(df))
    
exportDataframe(utrechtNecBB, 'Balbezit Utrecht-NEC')
exportDataframe(necCambuurBB, 'Balbezit NEC-NEC')
exportDataframe(pecNecBB, 'Balbezit Utrecht-NEC')
exportDataframe(NecRodaBB , 'Balbezit Utrecht-NEC')